<h2>This script is for internal developer testing.</h2>

In [ ]:
import numpy as np
from wfdb import readsignal
# Testing rdsamp format 212
sig, fields=readsignal.rdsamp('sampledata/100', sampfrom=100, sampto=9999) # Default physical values
print(sig)
print(len(sig))

sig, fields=readsignal.rdsamp('sampledata/100', physical=0) # Digital values
print(sig)

In [ ]:
# Testing rdsamp format 16
sig, fields=readsignal.rdsamp('sampledata/test01_00s')
print(sig)
print(fields)


In [ ]:
# Testing rdsamp format 80
sig, fields=readsignal.rdsamp('sampledata/3000003_0003')
print(sig)
print(fields)

In [ ]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
import numpy as np
from wfdb import readsignal
sig, fields=readsignal.rdsamp('sampledata/a103l') 
#print(sig)
#print(sig.shape)
#print(fields)

from wfdb import plotwfdb

plotwfdb.plotsigs(sig, fields)


# With channel selection and sampfrom sampto
#sig, fields=readsignal.rdsamp('sampledata/a103l', channels=[2,0], sampfrom=250, sampto=80000) 
#print('\n\n')
#print(sig)
#print(fields)

In [ ]:
# Testing rdsamp multi-segment variable layout record (from https://physionet.org/physiobank/database/mimic2wdb/matched/s00001/)
import numpy as np
from wfdb import readsignal

# Return list of arrays
sig, fields=readsignal.rdsamp('sampledata/matched/s00001/s00001-2896-10-10-00-31', sampfrom=3200, sampto=20000, stacksegments=0)
#print("len(sig): ", len(sig))
#print(len(fields))

print(sig)
print("\n\n")
print(sig[len(sig)-1])
print("\n\n")
print(len(sig[len(sig)-1]))
print("\n\n")
print(fields[0])
print("\n\n")
print(fields[1])
print("\n\n")
print(fields[2])

In [ ]:
# Testing rdsamp multi-segment variable layout record - stacksegments
import numpy as np
from wfdb import readsignal


# Return single numpy array
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', stacksegments=1, sampto=10000, channels=[1, 2])

print(sig)
print(sig.shape)
print("\n\n")
print(fields[0])
print("\n\n")
print(fields[1])
print("\n\n")
print(fields[2])

In [ ]:
# Testing rdsamp multi-dat signals (from https://physionet.org/physiobank/database/ptbdb/patient001/)
import numpy as np
from wfdb import readsignal

sig, fields=readsignal.rdsamp('sampledata/s0010_re', channels=[14, 0, 5, 10, 11], sampfrom=100, sampto=20000)
from wfdb import plotwfdb
plotwfdb.plotsigs(sig, fields)
print("\n\n\n", sig, "\n\n\n")
print(fields)
print(fields["signame"])

In [ ]:
# Testing rdsamp multi-segment record with sampfrom, sampto, and channel selection
import numpy as np
from wfdb import readsignal

# Return list of arrays
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=10, sampto=10200, channels=[2,3], stacksegments=0)

print(sig)
# Return single array
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=10, sampto=10200, channels=[1,0,2],stacksegments=1)

print(sig)

<h2>Decide comprehensive tests before adding to travis - signals</h2>
Target files created with wfdb software package's <code>rdsamp</code>

In [ ]:
import numpy as np
from wfdb import readsignal

# Test 1 - Format 212/Entire signal/Physical 
# Target file created with: rdsamp -r sampledata/100 -P | cut -f 2- > target1
sig, fields=readsignal.rdsamp('sampledata/100') 

# Test 2 - Format 212/Selected Duration/Selected Channel/Digital
# Target file created with: rdsamp -r sampledata/100 -f 1 -t 30 -s 1 | cut -f 2- > target2
sig, fields=readsignal.rdsamp('sampledata/100', sampfrom=360, sampto=10800, channels=[1], physical=0)  

# Test 3 - Format 16/Entire signal/Digital
# Target file created with: rdsamp -r sampledata/test01_00s | cut -f 2- > target3
sig, fields=readsignal.rdsamp('sampledata/test01_00s', physical=0)

# Test 4 - Format 16 with byte offset/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/a103l -f 50 -t 160 -s 2 0 -P | cut -f 2- > target4
sig, fields=readsignal.rdsamp('sampledata/a103l', sampfrom=12500, sampto=40000, channels=[2, 0])

# Test 5 - Format 16 with byte offset/Selected Duration/Selected Channels/Digital
# Target file created with: rdsamp -r sampledata/a103l -f 80 -s 0 1 | cut -f 2- > target5
sig, fields=readsignal.rdsamp('sampledata/a103l', sampfrom=20000, physical=0, channels=[0, 1])

# Test 6 - Format 80/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/3000003_0003 -f 1 -t 8 -s 1 -P | cut -f 2- > target6
sig, fields=readsignal.rdsamp('sampledata/3000003_0003', sampfrom=125, sampto=1000, channels=[1])

# Test 7 - Multi-dat/Entire signal/Digital
# Target file created with: rdsamp -r sampledata/s0010_re | cut -f 2- > target7
sig, fields=readsignal.rdsamp('sampledata/s0010_re', physical=0)

# Test 8 - Multi-dat/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/s0010_re -f 5 -t 38 -P -s 13 0 4 8 3 | cut -f 2- > target8
sig, fields=readsignal.rdsamp('sampledata/s0010_re', sampfrom=5000, sampto=38000, channels=[13, 0, 4, 8, 3])

<h2>Decide comprehensive tests before adding to travis - annotations</h2>
Target files created with wfdb software package's <code>rdann</code>

In [ ]:
import numpy as np
from wfdb import readannot

# Test 1 - 
# Target file created with: rdann -r 100 -a atr > anntarget1
annsamp, anntype, num, subtype, chan, aux, annfs=readannot.rdann('sampledata/100', 'atr')


# Test 2
# Target file created with: rdann -r chb01_03.edf -a seizures > anntarget2
annsamp, anntype, num, subtype, chan, aux, annfs=readannot.rdann('sampledata/chb01_03', 'seizures')


In [ ]:
from wfdb import readannot
annsamp, anntype, num, subtype, chan, aux, annfs=readannot.rdann('sampledata/100', 'fake2', sampfrom=17)
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)


a=aux[3]
print(a)
print(len(a))
a=aux[5]
print(a)
print(len(a))

In [ ]:
from wfdb import readannot
annsamp, anntype, num, subtype, chan, aux, annfs=readannot.rdann('sampledata/chb01_03.edf', 'seizures', sampfrom=766977, sampto=777326)
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)

In [ ]:
from wfdb import readannot
annsamp, anntype, num, subtype, chan, aux, annfs=readannot.rdann('sampledata/100', 'fake3', anndisp=1)
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)

In [ ]:
import re
import numpy as np
from wfdb import readannot

# Read data from WFDB python package
annsamp, anntype, subtype, chan, num, aux, annfs=readannot.rdann('sampledata/100', 'atr')
aux[0]='(N' # This is not the fault of the script. The annotation file specifies a length 3 
# aux field with a null written after '(N' which the script correctly picks up. I am just 
# getting rid of the null in this unit test to compare with the regexp output below which has 
# no null to detect in the output text file of rdann. 

# Target data from WFDB software package
lines=tuple(open('tests/targetoutputdata/anntarget1', 'r'))
nannot=len(lines)

Ttime=[None]*nannot
Tannsamp=np.empty(nannot, dtype='object')
Tanntype=[None]*nannot #np.chararray(nannot)
Tnum=np.empty(nannot, dtype='object')
Tsubtype=np.empty(nannot, dtype='object')
Tchan=np.empty(nannot, dtype='object')
Taux=[None]*nannot

RXannot=re.compile('[ \t]*(?P<time>[\w\.:]+) +(?P<annsamp>\d+) +(?P<anntype>.) +(?P<subtype>\d+) +(?P<chan>\d+) +(?P<num>\d+)\t?(?P<aux>.*)')

for i in range(0, nannot):
    Ttime[i], Tannsamp[i], Tanntype[i], Tsubtype[i], Tchan[i], Tnum[i], Taux[i]=RXannot.findall(lines[i])[0]
        
# Convert objects into integers
Tannsamp=Tannsamp.astype('int')
Tnum=Tnum.astype('int')
Tsubtype=Tsubtype.astype('int')
Tchan=Tchan.astype('int')

# Compare
print(np.array_equal(annsamp, Tannsamp)) # true
print(np.array_equal(anntype, Tanntype))  # true
print(np.array_equal(num, Tnum)) # false
print(np.array_equal(subtype, Tsubtype)) # false
print(np.array_equal(chan, Tchan)) # true
print(aux==Taux) #false



In [ ]:
import re
import numpy as np
from wfdb import readannot

# Read data from WFDB python package
annsamp, anntype, subtype, chan, num, aux, annfs=readannot.rdann('sampledata/12726', 'anI')

# Target data from WFDB software package
lines=tuple(open('tests/targetoutputdata/anntarget2', 'r'))
nannot=len(lines)

Ttime=[None]*nannot
Tannsamp=np.empty(nannot, dtype='object')
Tanntype=[None]*nannot 
Tsubtype=np.empty(nannot, dtype='object')
Tchan=np.empty(nannot, dtype='object')
Tnum=np.empty(nannot, dtype='object')
Taux=[None]*nannot

RXannot=re.compile('[ \t]*(?P<time>[\[\]\w\.:]+) +(?P<annsamp>\d+) +(?P<anntype>.) +(?P<subtype>\d+) +(?P<chan>\d+) +(?P<num>\d+)\t?(?P<aux>.*)')

for i in range(0, nannot):
    Ttime[i], Tannsamp[i], Tanntype[i], Tsubtype[i], Tchan[i], Tnum[i], Taux[i]=RXannot.findall(lines[i])[0]

# Convert objects into integers
Tannsamp=Tannsamp.astype('int')
Tnum=Tnum.astype('int')
Tsubtype=Tsubtype.astype('int')
Tchan=Tchan.astype('int')

# Compare
comp=[np.array_equal(annsamp, Tannsamp), np.array_equal(anntype, Tanntype), np.array_equal(subtype, Tsubtype), np.array_equal(chan, Tchan), np.array_equal(num, Tnum), aux==Taux] 

print(comp)

In [ ]:
print(chan)
print(Tchan)

In [ ]:
# For writing test annotation files based on first 3 annotations of 12726.anI

f=open('tmp.fake', 'wb')

indata= np.array([0, 236,   1,   0, 200,  84,   0,  88, 255, 251,  21, 252,  73, 110, 105, 116,
                  105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
                  112,   0,   0, 236,   0,   0,  67,  50,   0,  88,  21, 252,  67, 111, 110,  99,
108, 117, 100, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 117,
112,   0,   0, 236,   0,   0, 114, 183,   0,  88,  23, 252,  73, 110, 105, 116,
105,  97, 116, 101,  32, 115, 108, 111, 119,  32, 116, 105, 108, 116,  32, 100,
111, 119, 110,   0, 0 ,0 ]).astype('u1')


print(indata)

indata.tofile(f)


f.close()


In [ ]:
import re
import numpy as np
from wfdb import readannot

# Read data from WFDB python package
#annsamp, anntype, subtype, chan, num, aux, annfs=readannot.rdann('tmp', 'fake')
readannot.rdann('tmp', 'fake')

In [ ]:
import numpy as np
x=['hi','2']

x.index('hi')